## Segmentation using Slicer 

Segmentation Editor example using segmentation editor
perform segmentation on volume node 

In [ ]:
import os
import vtk, qt, ctk, slicer

In [ ]:
# Get VolumeNode
volumeNode = slicer.util.getNode('Node name')

# Create segmentation
segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
segmentationNode.CreateDefaultDisplayNodes() # only needed for display
segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(volumeNode)
addedSegmentID = segmentationNode.GetSegmentation().AddEmptySegment('Node name' + "_segment")
   
# Create segment editor to get access to effects
segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
segmentEditorNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentEditorNode")
segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)
segmentEditorWidget.setSegmentationNode(segmentationNode)
segmentEditorWidget.setMasterVolumeNode(volumeNode)

# Thresholding
segmentEditorWidget.setActiveEffectByName("Threshold")
effect = segmentEditorWidget.activeEffect()
effect.setParameter("MinimumThreshold", -100)
#effect.setParameter("MaximumThreshold", self.thresholdMax)
effect.self().onApply()

# Islands
segmentEditorWidget.setActiveEffectByName("Islands")
effect = segmentEditorWidget.activeEffect()
effect.setParameter("Operation", "KEEP_LARGEST_ISLAND")            
effect.self().onApply()

#Erode
segmentEditorWidget.setActiveEffectByName("Margin")
effect = segmentEditorWidget.activeEffect()
effect.setParameter("MarginSizeMm", -1.00)            
effect.self().onApply()

#Dialate
segmentEditorWidget.setActiveEffectByName("Margin")
effect = segmentEditorWidget.activeEffect()
effect.setParameter("MarginSizeMm", 1.00)            
effect.self().onApply()

#HoleFilling
segmentEditorWidget.setActiveEffectByName("Smoothing")
effect = segmentEditorWidget.activeEffect()
effect.setParameter("SmoothingMethod", "CLOSING")
effect.setParameter("KernalSizeMm", self.holeFill)
effect.self().onApply()

# Smoothing
segmentEditorWidget.setActiveEffectByName("Smoothing")
effect = segmentEditorWidget.activeEffect()
effect.setParameter("SmoothingMethod", "MEDIAN")
effect.setParameter("KernelSizeMm", 3)
effect.self().onApply()

## Segmentation using openCV

perform segmentation on numpy array

In [ ]:
import cv2
from scipy import ndimage
import numpy as np

In [ ]:
# threshold 
ret, img_th = cv2.threshold(arr, threshold, 1, 0)

# remain largest region
label_im, nb_labels = ndimage.label(img_th)
sizes = ndimage.sum(img_th, label_im, range(nb_labels + 1))
mask = sizes == np.max(sizes)
label_img = mask[label_im]

kernel = np.ones((3, 3), np.uint8)
#MORPH_OPEN
open_volume = cv2.morphologyEx(np.array(label_img).astype(np.uint8), cv2.MORPH_OPEN, kernel)

#MORPH_Close
closed_volume = cv2.morphologyEx(open_volume, cv2.MORPH_CLOSE, kernel)

# fill holes 
temp = np.zeros_like(label_img)
for idx in range(closed_volume.shape[0]):
    closed_slice = closed_volume[idx, :, :]
    mask_slice = ndimage.morphology.binary_fill_holes(closed_slice)
    temp[idx, :, :] = mask_slice

# Erode
temp = ndimage.binary_erosion(temp)
mask_volume = np.array(temp)
